In [ ]:
!pip install cohere
!pip install llama-index transformers accelerate bitsandbytes

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
import torch
from llama_index import ServiceContext
from llama_index import VectorStoreIndex
from llama_index import SummaryIndex
from llama_index.response.notebook_utils import display_response


Reading documents from data folder

In [ ]:
from llama_index.readers import BeautifulSoupWebReader
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

Loading llama 2 7b-model from Huggingface

In [ ]:

from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

# huggingface api token
hf_token = "..."

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

Creating document index

In [ ]:

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

Let's do document query

In [ ]:

query_engine = vector_index.as_query_engine()

response = query_engine.query("Can you tell results on paper 'Attention Is All You Need'?")
display_response(response)

response = query_engine.query("Whats new in MyTax?")
display_response(response)

**`Final Response:`** Based on the context information provided, the paper "Attention Is All You Need" by Minh-Thang Luong, Hieu Pham, and Christopher D. Manning (2015) presents an effective approach to attention-based neural machine translation. The paper proposes a simple and efficient attention mechanism that replaces traditional recurrent neural network (RNN) architectures with attention mechanisms, resulting in better translation performance.

The paper's key contributions include:

1. Scaled dot-product attention: The authors propose a scaled dot-product attention mechanism that computes the attention weights by taking the dot product of the query and key vectors, scaling the result by √dk, and applying a softmax function. This mechanism allows the model to attend to different parts of the input sequence simultaneously and improve translation performance.
2. Multi-head attention: The authors propose a multi-head attention mechanism that allows the model to jointly attend to information from different representation subspaces at different positions. This is achieved by linearly projecting the queries, keys, and values onto different dimensions and performing the attention function in parallel. The outputs of each attention head are concatenated and projected back to the original dimension, resulting

**`Final Response:`** Based on the provided context information, here are the key new features and improvements in MyTax as of November 2023:

1. Improved transaction search: The search terms for transaction search have been improved, allowing users to include payments made in the search period in the search results.
2. New views for accountants: Accounting firms and accountants will have access to a new view in MyTax, which will display clients' open balances, latest summaries, and requests for additional information awaiting a response, as well as notifications of any overdue tax returns.
3. Ability for EU citizens to log in with eIDAS identification: Individuals and companies based in other EU countries can log in to MyTax using eIDAS identification methods issued by their own country, enabling them to send a contact request and file certain requests and returns in a limited scope.
4. Improved processing status of filings: Users will be able to follow the processing status of their filings, including Received, Being processed, Waiting for further information, and Finished.
5. Country-by-country reporting: Country-by-country reporting can be filed in My

For Finnish text let's use gpt-4 instead

In [ ]:
import os
import openai
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index import VectorStoreIndex
from llama_index import SummaryIndex

#OpenAI api key
os.environ['OPENAI_API_KEY'] = "..."
openai.api_key = os.environ['OPENAI_API_KEY']

llm = OpenAI(model="gpt-4", temperature=0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)
query_engine = vector_index.as_query_engine()

response = query_engine.query("Voitko kertoa ranskalaisin viivoin mitä uutta OmaVerossa on?")
display_response(response)

**`Final Response:`** - EU-maiden henkilö- ja yritysasiakkaat voivat käyttää eIDAS-tunnistautumista OmaVerossa.
- Verotuksen maakohtaisen raportoinnin ilmoitukset voi antaa OmaVerossa 19.11. alkaen.
- Erillinen kirjautumissivu Verohallinnon asiointipalveluun poistuu käytöstä ja toimintoihin pääsee OmaVerossa.
- OmaVerossa voi antaa tiettyjä vuosi-ilmoituksia, kiinteistönvälittäjän tai arvopaperikauppiaan varainsiirtoveroilmoituksen, lähettää perukirjan tai tehdä muita perukirjaan liittyviä pyyntöjä asiakkaan puolesta, tehdä tietopyyntöjä yksittäisistä asiakirjoista, pyytää ennakollista keskustelua, hakea lähdeverokorttia tai hakea lähdeveron palautusta.
- Tapahtumahaun hakuehtoja on paranneltu.
- Tilitoimistoille ja kirjanpitäjille tulee OmaVerossa uusi näkymä, johon pääsee asiakasvalintalistalta.
- Ulkomaalainen henkilöasiakas voi täyttää yhteydenottopyynnön OmaVerossa.
- Ulkomaalaiset henkilö- ja yritysasiakkaat voivat antaa ilmoituksen tai tehdä hakemuksen OmaVerossa rajoitetusti.
- Maakohtaisen raportoinnin ilmoitukset voi antaa OmaVerossa 19.11. klo 18 alkaen.
- Lomake.fi sulkeutuu 29.2.2024, mutta maakohtaisen raportin ja ilmoituksen selvitysvelvollisesta voi antaa OmaVerossa jo 19.11. alkaen.

Let's imitate customer service

In [ ]:
response = query_engine.query("Haluaisin liittymän, jossa on mahdollisimman pieni kk-maksu. Voitko kertoa mikä DNA:n liittymä olisi kannatavin?")
display_response(response)

**`Final Response:`** DNA Puhe Plus -liittymä olisi kannattavin valinta, sillä sen kuukausimaksu on 7,90 €, mikä on pienin mainituista liittymistä.

We could use separate file to guide chat bot (here DNA_LMM_info.txt)

In [ ]:
response = query_engine.query("Voinko tilata liittymän tästä?")
display_response(response)

**`Final Response:`** Valitettavasti et voi tilata liittymää tästä. Sinun tulee ottaa yhteyttä asiakaspalveluun.